# QbraidProvider

In [1]:
import qbraid

from qbraid.providers import QbraidProvider

In [2]:
qbraid.__version__

'0.5.0.dev'

In [3]:
provider = QbraidProvider()

provider.get_devices()

[Device('name': Aquila, 'arn': arn:aws:braket:us-east-1::device/qpu/quera/Aquila),
 Device('name': Aria 1, 'arn': arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1),
 Device('name': Aria 2, 'arn': arn:aws:braket:us-east-1::device/qpu/ionq/Aria-2),
 Device('name': Aspen-M-3, 'arn': arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-3),
 Device('name': Forte 1, 'arn': arn:aws:braket:us-east-1::device/qpu/ionq/Forte-1),
 Device('name': Harmony, 'arn': arn:aws:braket:us-east-1::device/qpu/ionq/Harmony),
 Device('name': Lucy, 'arn': arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy),
 Device('name': SV1, 'arn': arn:aws:braket:::device/quantum-simulator/amazon/sv1),
 Device('name': TN1, 'arn': arn:aws:braket:::device/quantum-simulator/amazon/tn1),
 Device('name': dm1, 'arn': arn:aws:braket:::device/quantum-simulator/amazon/dm1),
 <IBMBackend('simulator_stabilizer')>,
 <IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_kyoto')>,
 <IBMBackend('ibm_osaka')>,
 <IBMBackend('ibmq_qasm_simulator')>,
 <IB

In [4]:
aws_device = provider.get_device("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")

type(aws_device)

braket.aws.aws_device.AwsDevice

In [5]:
ibm_device = provider.get_device("ibm_brisbane")

type(ibm_device)

qiskit_ibm_provider.ibm_backend.IBMBackend

## BraketProvider

In [6]:
from qbraid.providers.aws import BraketProvider, BraketDevice, get_quantum_task_cost

braket_provider = BraketProvider()

### Query AWS devices

In [7]:
braket_provider.get_devices(statuses="ONLINE", types=["QPU"])

[Device('name': Aquila, 'arn': arn:aws:braket:us-east-1::device/qpu/quera/Aquila),
 Device('name': Forte 1, 'arn': arn:aws:braket:us-east-1::device/qpu/ionq/Forte-1),
 Device('name': Lucy, 'arn': arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy)]

In [8]:
braket_device = BraketDevice(aws_device)

braket_device.vendor, braket_device.provider, braket_device.name

('AWS', 'Oxford', 'Lucy')

In [9]:
braket_device.queue_depth()

8

In [10]:
_, _, next_availability_switch_utc = braket_device.availability_window()

print(next_availability_switch_utc)

2024-01-11T10:00:00Z


### Query AWS quantum tasks

In [11]:
tagged_tasks = braket_provider.get_tasks_by_tag("test")

len(tagged_tasks)

5

In [12]:
task_arn = tagged_tasks[0]

get_quantum_task_cost(task_arn)

Decimal('0.0037500000')

In [13]:
qbraid.get_jobs(filters={"tags": {"test": "123"}})

In [14]:
qbraid.get_jobs(filters={"vendorJobId": task_arn})

In [15]:
qbraid_job = qbraid.job_wrapper("aws_dm_sim-markovshama-qjob-e5vxqgdjsx23tuzn1ryw")

type(qbraid_job)

qbraid.providers.aws.job.BraketQuantumTask

In [16]:
qbraid_job.status()

<JobStatus.COMPLETED: 'job has successfully run'>

In [17]:
qbraid_job.get_cost()

0.00375

## QiskitProvider

In [18]:
from qbraid.providers.ibm import QiskitProvider, QiskitBackend

provider = QiskitProvider()

In [19]:
provider.get_devices(operational=True, simulator=False)

[<IBMBackend('ibm_brisbane')>,
 <IBMBackend('ibm_kyoto')>,
 <IBMBackend('ibm_osaka')>]

In [20]:
provider.ibm_least_busy_qpu()

'ibm_q_kyoto'

### Query IBM backends

In [21]:
qiskit_backend = QiskitBackend(ibm_device)

qiskit_backend.provider, qiskit_backend.name

('IBM', 'ibm_brisbane')

In [22]:
qiskit_backend.status()

<DeviceStatus.ONLINE: 0>

In [23]:
qiskit_backend.queue_depth()

65

### Query IBM jobs

In [24]:
qbraid.get_jobs(
    filters={
        "tags": {"test": "*"},
        "qbraidDeviceId": QiskitProvider.ibm_to_qbraid_id("ibmq_qasm_simulator"),
    }
)

In [25]:
qbraid_job = qbraid.job_wrapper(
    "ibm_q_qasm_simulator-markovshama-qjob-zy12imhp7n43what3zeh"
)

type(qbraid_job)

qbraid.providers.ibm.job.QiskitJob

In [26]:
qbraid_job.status()

<JobStatus.COMPLETED: 'job has successfully run'>